In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from jsputils import paths, classes, nsdorg, plotting, encoding, nnutils, selectivity, eiganalysis, gpu_encoding
import os
from os.path import exists
import cortex
import numpy as np
import matplotlib.pyplot as plt
from fastprogress import progress_bar
import scipy.stats as stats
from scipy.spatial.distance import pdist, squareform
import copy
import time
import gc
from IPython.core.debugger import set_trace
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from PIL import Image

import sys
sys.path.append('/home/jovyan/work/DropboxSandbox/GSN')
import gsn
from gsn.rsa_noise_ceiling import rsa_noise_ceiling

import torch
import torchvision
import torchlens as tl

from torchvision.transforms._presets import ImageClassification

from sklearn.linear_model import Lasso, LinearRegression

In [ ]:
nsddir = paths.nsd()

feature_savedir = f'{os.getcwd()}/analysis_outputs/3d-LowLevel'
encoding_savedir =  f'{os.getcwd()}/analysis_outputs/3-Encoding'

train_imageset = 'nonshared1000-3rep-batch0'
val_imageset = 'nonshared1000-3rep-batch1'
test_imageset = 'special515'

subjs = [f'subj0{s}' for s in range(1,9)]

In [ ]:
for subj in progress_bar(subjs):
    
    subj_image_data = nsdorg.get_NSD_train_test_images(subj, train_imageset, test_imageset)
    
    if subj == 'subj01':
        
        savefn = f'{feature_savedir}/special515_images.mat'
        if not exists(savefn):
            # Save it as a .mat file
            sio.savemat(savefn, {'images': subj_image_data['test']})
            #np.save(f'{savedir}/special515_images.npy', subj_image_data['test'])
            print(subj_image_data['test'].shape)
        else:
            print('skipping, already exists')
    
    savefn = f'{feature_savedir}/{subj}_train-{train_imageset}_images.mat'
    if not exists(savefn):
        sio.savemat(savefn, {'images': subj_image_data['train']})
        #np.save(savefn, subj_image_data['train'])
        print(subj, subj_image_data['train'].shape)
    else:
        print('skipping, already exists')

In [ ]:
roi_list = ['FFA-1','FFA-2','OFA','PPA','OPA','EBA','FBA-1','FBA-2','VWFA-1','VWFA-2','OWFA']

train_imageset = 'nonshared1000-3rep-batch0'
val_imageset = 'nonshared1000-3rep-batch1'
test_imageset = 'special515'

space = 'nativesurface'
beta_version = 'betas_fithrf_GLMdenoise_RR'
ncsnr_threshold = 0.3


In [ ]:
def load_gist_gabor_features(feature_savedir, subj, ft):


    if not 'subj' in subj:
        struct = sio.loadmat(f'{feature_savedir}/{test_imageset}_{ft}.mat')
    else:
        struct = sio.loadmat(f'{feature_savedir}/{subj}_{ft}.mat')

    # Access the struct as a dictionary
    struct = struct[ft]

    # If the .mat file was saved in MATLAB with the -v7.3 flag,
    # the loaded data is a structured numpy array. To convert it into a dictionary:
    if not isinstance(struct, dict):
        struct = {n: struct[n][0,0] for n in struct.dtype.names}
    
    return struct['featureMatrix']

print(load_gist_gabor_features(feature_savedir, 'special515','GistPC').shape)


In [ ]:
results = dict()

for ft in ['GistPC','Gabor']:
    
    results[ft] = dict()
    
    test_features = load_gist_gabor_features(feature_savedir, 'special515', ft)

    for roi in roi_list:

        results[ft][roi] = dict()

        for subj in progress_bar(subjs):
            NSDsubj = classes.fMRISubject(subj, space, beta_version)
            ROI = classes.BrainRegion(NSDsubj, roi)

            ROI.load_betas()
            ROI.get_ncsnr_mask(threshold = ncsnr_threshold)
            ROI.load_encoding_data(train_imageset, val_imageset, test_imageset)
            
            train_features = load_gist_gabor_features(feature_savedir, subj, ft)

            encoder = classes.EncodingProcedureGistGabor(ROI, ft, 
                                                         train_features, 
                                                         test_features,
                                                         method = 'ols', 
                                                         positive = False,
                                                         alphas = [None])  # 0.001 for untrained. 0.1 for trained



            encoder.encode_features(encoding_savedir, 
                                  overwrite = False)

            results[ft][roi][subj] = encoder.results_df